# Bonus Lab — Powers of Ten Drill
## Timed rounds to build prefix fluency for circuit calculations

---

**Prerequisites:** Module 00 (Electrical Fundamentals)

**Format:** Run each cell in order. Each round generates fresh, randomized problems and times you. Re-run any round cell for new problems and another attempt.

**How to answer:** Type your answer and press Enter. Accepted formats:
- With prefix: `4.7k`, `4.7kΩ`, `4.7 kΩ`, `500uA`, `500µA`
- Scientific notation: `4.7e3`, `5e-4`
- Plain number: `4700`, `0.0005`
- For prefix name questions: `micro`, `µ`, `10^-6`

**Estimated time:** 20–30 minutes

In [ ]:
# Setup — run this cell first
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import re
import math
%matplotlib inline

plt.rcParams.update({
    'figure.figsize': (10, 5),
    'axes.grid': True,
    'font.size': 12,
    'lines.linewidth': 2,
    'grid.alpha': 0.3,
})

# ---- SI prefix tables ----

PREFIX_TO_EXP = {
    'T': 12, 'G': 9, 'M': 6, 'k': 3,
    '': 0,
    'm': -3, 'u': -6, '\u00b5': -6, 'n': -9, 'p': -12,
}

EXP_TO_PREFIX = {
    12: 'T', 9: 'G', 6: 'M', 3: 'k',
    0: '',
    -3: 'm', -6: '\u00b5', -9: 'n', -12: 'p',
}

PREFIX_NAMES = {
    'T': 'tera', 'G': 'giga', 'M': 'mega', 'k': 'kilo',
    'm': 'milli', 'u': 'micro', '\u00b5': 'micro',
    'n': 'nano', 'p': 'pico',
}

NAME_TO_PREFIX = {
    'tera': 'T', 'giga': 'G', 'mega': 'M', 'kilo': 'k',
    'milli': 'm', 'micro': '\u00b5', 'nano': 'n', 'pico': 'p',
}

# Unit aliases
UNIT_ALIASES = {
    'ohm': '\u03a9', 'ohms': '\u03a9', '\u03a9': '\u03a9',
    'a': 'A', 'amp': 'A', 'amps': 'A',
    'v': 'V', 'volt': 'V', 'volts': 'V',
    'f': 'F', 'farad': 'F', 'farads': 'F',
    'h': 'H', 'henry': 'H', 'henrys': 'H', 'henries': 'H',
    'w': 'W', 'watt': 'W', 'watts': 'W',
    's': 's', 'sec': 's', 'second': 's', 'seconds': 's',
    'hz': 'Hz', 'hertz': 'Hz',
}

# ---- Answer parsing ----

def normalize_value(text):
    """Parse a text answer into a numeric value.
    Handles: '4.7k', '4.7k\u03a9', '4700', '4.7e3', '0.5mA', '500\u00b5A', etc.
    Returns float or None if unparseable.
    """
    text = text.strip()
    if not text:
        return None

    # Try pure numeric / scientific notation first
    try:
        return float(text)
    except ValueError:
        pass

    # Pattern: optional number, optional prefix, optional unit
    # Examples: 4.7k, 4.7k\u03a9, 0.5mA, 500\u00b5A, 10ms, .5mA
    pattern = r'^([+-]?(?:\d+\.?\d*|\.\d+)(?:[eE][+-]?\d+)?)\s*([TGMkmu\u00b5npKU]?)\s*(.*)$'
    m = re.match(pattern, text)
    if m:
        num_str, prefix, unit_str = m.groups()
        try:
            num = float(num_str)
        except ValueError:
            return None

        # Normalize prefix
        if prefix == 'K':  # common mistake: capital K for kilo
            prefix = 'k'
        if prefix == 'U':  # common shorthand for micro
            prefix = 'u'

        exp = PREFIX_TO_EXP.get(prefix, 0)
        return num * (10 ** exp)

    return None


def format_eng(value, unit=''):
    """Format a value in engineering notation with SI prefix."""
    if value == 0:
        return f'0{unit}'

    exp = int(math.floor(math.log10(abs(value))))
    # Round to nearest multiple of 3
    eng_exp = (exp // 3) * 3
    # Clamp to known prefixes
    eng_exp = max(-12, min(12, eng_exp))

    mantissa = value / (10 ** eng_exp)
    prefix = EXP_TO_PREFIX.get(eng_exp, f'e{eng_exp}')

    # Format mantissa nicely
    if mantissa == int(mantissa):
        return f'{int(mantissa)}{prefix}{unit}'
    else:
        # Remove trailing zeros
        formatted = f'{mantissa:.4g}'
        return f'{formatted}{prefix}{unit}'


def check_numeric(answer_text, expected, tolerance=0.02):
    """Check if answer_text evaluates to a value within tolerance of expected."""
    val = normalize_value(answer_text)
    if val is None:
        return False
    if expected == 0:
        return abs(val) < 1e-15
    return abs(val - expected) / abs(expected) <= tolerance


# ---- Session state ----

session_results = []


def run_round(problems, round_name):
    """Run a timed drill round.

    Each problem is a dict with:
      'question': str to display
      'check': callable(answer_text) -> bool
      'answer_display': str to show if wrong
    """
    n = len(problems)
    print(f'\n{"=" * 50}')
    print(f'{round_name}  \u2014  {n} problems')
    print(f'{"=" * 50}\n')

    score = 0
    streak = 0
    best_streak = 0
    start = time.time()

    for i, p in enumerate(problems, 1):
        ans = input(f'  [{i}/{n}] {p["question"]}  \u2192  ')
        if p['check'](ans):
            score += 1
            streak += 1
            best_streak = max(best_streak, streak)
            print(f'         \u2713 Correct!')
        else:
            streak = 0
            print(f'         \u2717 Answer: {p["answer_display"]}')

    elapsed = time.time() - start
    accuracy = score / n * 100

    print(f'\n{"\u2500" * 40}')
    print(f'  Score:        {score}/{n} ({accuracy:.0f}%)')
    print(f'  Time:         {elapsed:.1f}s ({elapsed/n:.1f}s per problem)')
    print(f'  Best streak:  {best_streak}')
    print(f'{"\u2500" * 40}\n')

    session_results.append({
        'round': round_name,
        'score': score,
        'total': n,
        'accuracy': accuracy,
        'time': elapsed,
        'per_problem': elapsed / n,
        'best_streak': best_streak,
    })

    return score, n


print('Setup complete. Ready to drill!')

---
## The Prefix Ladder

Each step is \u00d71000 (three orders of magnitude):

| Prefix | Symbol | Power | Electrical example |
|--------|--------|-------|--------------------|
| pico   | p      | 10\u207b\u00b9\u00b2 | 22pF ceramic cap |
| nano   | n      | 10\u207b\u2079  | 100nF bypass cap |
| micro  | \u00b5      | 10\u207b\u2076  | 470\u00b5F electrolytic cap |
| milli  | m      | 10\u207b\u00b3  | 20mA LED current |
| \u2014      | \u2014      | 10\u2070   | 5V supply, 220\u03a9 resistor |
| kilo   | k      | 10\u00b3   | 10k\u03a9 pull-up resistor |
| mega   | M      | 10\u2076   | 1M\u03a9 scope input impedance |

### The key insight: prefix products

When you multiply prefixed quantities, **add the exponents**:

- k\u03a9 \u00d7 \u00b5F = 10\u00b3 \u00d7 10\u207b\u2076 = 10\u207b\u00b3 \u2192 **ms** (milliseconds)
- k\u03a9 \u00d7 nF = 10\u00b3 \u00d7 10\u207b\u2079 = 10\u207b\u2076 \u2192 **\u00b5s** (microseconds)
- M\u03a9 \u00d7 pF = 10\u2076 \u00d7 10\u207b\u00b9\u00b2 = 10\u207b\u2076 \u2192 **\u00b5s**
- V \u00f7 k\u03a9 = 10\u2070 \u00f7 10\u00b3 = 10\u207b\u00b3 \u2192 **mA**
- V \u00f7 M\u03a9 = 10\u2070 \u00f7 10\u2076 = 10\u207b\u2076 \u2192 **\u00b5A**

Memorize these five and most prefix arithmetic becomes instant.

---
## Component Marking Conventions

**Capacitor 3-digit code:** Two significant digits + number of trailing zeros, in **picofarads**.
- `104` = 10 \u00d7 10\u2074 pF = 100,000pF = 100nF = 0.1\u00b5F
- `473` = 47 \u00d7 10\u00b3 pF = 47,000pF = 47nF
- `222` = 22 \u00d7 10\u00b2 pF = 2,200pF = 2.2nF

**Resistor shorthand:** On datasheets, `4K7` = 4.7k\u03a9, `2R2` = 2.2\u03a9, `1M0` = 1M\u03a9. The letter replaces the decimal point.

In [ ]:
# ---- Round 1: Prefix Recognition ----

def make_round1():
    problems = []

    # Type A: "What does prefix X mean?"
    prefixes_a = random.sample(
        [('p', 'pico', -12), ('n', 'nano', -9), ('\u00b5', 'micro', -6),
         ('m', 'milli', -3), ('k', 'kilo', 3), ('M', 'mega', 6)], 3)
    for sym, name, exp in prefixes_a:
        def check(ans, name=name, exp=exp):
            ans = ans.strip().lower()
            return (ans == name or ans == f'10^{exp}'
                    or ans == f'1e{exp}' or ans == f'10^({exp})')
        problems.append({
            'question': f'What does the prefix "{sym}" mean? (name or power of 10)',
            'check': check,
            'answer_display': f'{name} (10^{exp})',
        })

    # Type B: "What prefix is 10^X?"
    prefixes_b = random.sample(
        [(-12, 'p', 'pico'), (-9, 'n', 'nano'), (-6, '\u00b5', 'micro'),
         (-3, 'm', 'milli'), (3, 'k', 'kilo'), (6, 'M', 'mega')], 2)
    for exp, sym, name in prefixes_b:
        def check(ans, sym=sym, name=name):
            ans = ans.strip()
            ans_lower = ans.lower()
            return ans in (sym, 'u') and True or ans_lower == name or ans == sym
        # More robust check:
        def check(ans, sym=sym, name=name):
            a = ans.strip()
            return (a == sym or a.lower() == name
                    or (sym == '\u00b5' and a.lower() in ('u', 'mu', 'micro'))
                    or (sym == 'M' and a == 'M')
                    or (sym == 'm' and a == 'm'))
        problems.append({
            'question': f'What prefix symbol represents 10^{exp}?',
            'check': check,
            'answer_display': f'{sym} ({name})',
        })

    # Type C: "Express X in base units"
    conversions = random.sample([
        ('10k\u03a9 in ohms', 10e3, '\u03a9'),
        ('4.7k\u03a9 in ohms', 4.7e3, '\u03a9'),
        ('220k\u03a9 in ohms', 220e3, '\u03a9'),
        ('0.001A using a prefix', 1e-3, 'A'),
        ('0.000047F using a prefix', 47e-9, 'F'),
        ('2,200,000\u03a9 using a prefix', 2.2e6, '\u03a9'),
        ('0.022A using a prefix', 22e-3, 'A'),
        ('0.0000001F using a prefix', 100e-9, 'F'),
    ], 3)
    for desc, expected, unit in conversions:
        def check(ans, expected=expected):
            return check_numeric(ans, expected)
        problems.append({
            'question': f'Express {desc}',
            'check': check,
            'answer_display': format_eng(expected, unit),
        })

    random.shuffle(problems)
    return problems

run_round(make_round1(), 'Round 1: Prefix Recognition')

In [ ]:
# ---- Round 2: Unit Conversion ----

def make_round2():
    problems = []

    # E12 resistor values (base values)
    e12 = [1.0, 1.2, 1.5, 1.8, 2.2, 2.7, 3.3, 3.9, 4.7, 5.6, 6.8, 8.2]
    # Common capacitor values
    cap_vals = [10, 22, 33, 47, 68, 100, 220, 330, 470]

    # Resistor: base -> prefixed
    for _ in range(3):
        base = random.choice(e12)
        mult, from_u, to_pref = random.choice([
            (1e3, '\u03a9', 'k\u03a9'), (1e6, '\u03a9', 'M\u03a9'), (1e3, 'k\u03a9', 'M\u03a9'),
        ])
        val_base = base * mult
        expected_prefixed = base
        def check(ans, expected=val_base):
            return check_numeric(ans, expected)
        # Decide direction
        if random.random() < 0.5:
            # prefixed -> base
            problems.append({
                'question': f'{base}{to_pref} = ? {from_u}',
                'check': check,
                'answer_display': f'{val_base:g}{from_u}',
            })
        else:
            # base -> prefixed
            def check2(ans, expected=base):
                return check_numeric(ans, expected)
            problems.append({
                'question': f'{val_base:g}{from_u} = ? {to_pref}',
                'check': check2,
                'answer_display': f'{base:g}{to_pref}',
            })

    # Capacitor: between prefix forms
    for _ in range(4):
        base_pf = random.choice(cap_vals)
        from_exp, to_exp = random.choice([
            (-12, -9),   # pF -> nF
            (-9, -6),    # nF -> \u00b5F
            (-6, -9),    # \u00b5F -> nF
            (-9, -12),   # nF -> pF
        ])
        # Scale base_pf to get a value in the 'from' prefix
        # base_pf is a base value; place it in the from_exp range
        val_si = base_pf * (10 ** from_exp)
        val_in_to = val_si / (10 ** to_exp)
        from_prefix = EXP_TO_PREFIX[from_exp]
        to_prefix = EXP_TO_PREFIX[to_exp]
        def check(ans, expected=val_in_to):
            return check_numeric(ans, expected)
        problems.append({
            'question': f'{base_pf}{from_prefix}F = ? {to_prefix}F',
            'check': check,
            'answer_display': f'{val_in_to:g}{to_prefix}F',
        })

    # Current conversions
    for _ in range(3):
        ma_val = random.choice([0.5, 1, 2, 5, 10, 20, 50, 100, 250, 500])
        if random.random() < 0.5:
            # mA -> A
            expected = ma_val * 1e-3
            def check(ans, expected=expected):
                return check_numeric(ans, expected)
            problems.append({
                'question': f'{ma_val:g}mA = ? A',
                'check': check,
                'answer_display': f'{expected:g}A',
            })
        else:
            # mA -> \u00b5A
            expected = ma_val * 1e3
            def check(ans, expected=expected):
                return check_numeric(ans, expected)
            problems.append({
                'question': f'{ma_val:g}mA = ? \u00b5A',
                'check': check,
                'answer_display': f'{expected:g}\u00b5A',
            })

    random.shuffle(problems)
    return problems

run_round(make_round2(), 'Round 2: Unit Conversion')

In [ ]:
# ---- Round 3: Component Markings ----

def make_round3():
    problems = []

    # 3-digit capacitor codes -> value
    # Code = AB x 10^C in picofarads
    cap_codes = [
        ('104', 100e-9, '100nF (0.1\u00b5F)'),
        ('103', 10e-9, '10nF'),
        ('473', 47e-9, '47nF'),
        ('222', 2.2e-9, '2.2nF'),
        ('471', 470e-12, '470pF'),
        ('105', 1e-6, '1\u00b5F'),
        ('224', 220e-9, '220nF (0.22\u00b5F)'),
        ('102', 1e-9, '1nF'),
        ('334', 330e-9, '330nF'),
        ('221', 220e-12, '220pF'),
    ]

    # Decode cap marking
    chosen = random.sample(cap_codes, 5)
    for code, val_si, display in chosen:
        # Ask in a random target unit
        target_exp = random.choice([-9, -6])  # nF or \u00b5F
        target_prefix = EXP_TO_PREFIX[target_exp]
        expected = val_si / (10 ** target_exp)
        def check(ans, expected=expected):
            return check_numeric(ans, expected)
        problems.append({
            'question': f'Capacitor marked "{code}" = ? {target_prefix}F',
            'check': check,
            'answer_display': f'{expected:g}{target_prefix}F ({display})',
        })

    # Encode value -> cap marking code
    encode_pool = [
        (22e-9, '223', '22nF'),
        (100e-9, '104', '100nF'),
        (4.7e-9, '472', '4.7nF'),
        (1e-6, '105', '1\u00b5F'),
        (330e-12, '331', '330pF'),
        (10e-9, '103', '10nF'),
    ]
    chosen_enc = random.sample(encode_pool, 3)
    for val_si, code, label in chosen_enc:
        def check(ans, code=code):
            return ans.strip() == code
        val_pf = val_si * 1e12
        problems.append({
            'question': f'What is the 3-digit code for {label}?',
            'check': check,
            'answer_display': code,
        })

    random.shuffle(problems)
    return problems

run_round(make_round3(), 'Round 3: Component Markings')

In [ ]:
# ---- Round 4: Ohm's Law with Prefixes ----

def make_round4():
    problems = []

    # V = IR, I = V/R, R = V/I  with prefixed values
    templates = [
        # (description_fn, compute_fn, answer_unit)
        # Find I given V and R
        lambda: _ohm_find_I(),
        # Find V given I and R
        lambda: _ohm_find_V(),
        # Find R given V and I
        lambda: _ohm_find_R(),
    ]

    for _ in range(10):
        fn = random.choice(templates)
        problems.append(fn())

    random.shuffle(problems)
    return problems


def _ohm_find_I():
    voltages = [1.5, 3.3, 5, 9, 12]
    resistors = [(100, '100\u03a9'), (220, '220\u03a9'), (470, '470\u03a9'),
                 (1e3, '1k\u03a9'), (2.2e3, '2.2k\u03a9'), (4.7e3, '4.7k\u03a9'),
                 (10e3, '10k\u03a9'), (47e3, '47k\u03a9'), (100e3, '100k\u03a9'),
                 (1e6, '1M\u03a9')]
    V = random.choice(voltages)
    R, R_str = random.choice(resistors)
    I = V / R
    I_display = format_eng(I, 'A')
    def check(ans, expected=I):
        return check_numeric(ans, expected)
    return {
        'question': f'{V}V across {R_str} \u2192 current?',
        'check': check,
        'answer_display': I_display,
    }


def _ohm_find_V():
    currents = [(1e-3, '1mA'), (2e-3, '2mA'), (5e-3, '5mA'),
                (10e-3, '10mA'), (20e-3, '20mA'), (50e-3, '50mA'),
                (100e-6, '100\u00b5A'), (500e-6, '500\u00b5A')]
    resistors = [(100, '100\u03a9'), (220, '220\u03a9'), (470, '470\u03a9'),
                 (1e3, '1k\u03a9'), (2.2e3, '2.2k\u03a9'), (4.7e3, '4.7k\u03a9'),
                 (10e3, '10k\u03a9')]
    I, I_str = random.choice(currents)
    R, R_str = random.choice(resistors)
    V = I * R
    V_display = format_eng(V, 'V')
    def check(ans, expected=V):
        return check_numeric(ans, expected)
    return {
        'question': f'{I_str} through {R_str} \u2192 voltage?',
        'check': check,
        'answer_display': V_display,
    }


def _ohm_find_R():
    voltages = [1.5, 3.3, 5, 9, 12]
    currents = [(0.5e-3, '0.5mA'), (1e-3, '1mA'), (2e-3, '2mA'),
                (2.5e-3, '2.5mA'), (5e-3, '5mA'), (10e-3, '10mA'),
                (20e-3, '20mA'), (50e-3, '50mA')]
    V = random.choice(voltages)
    I, I_str = random.choice(currents)
    R = V / I
    R_display = format_eng(R, '\u03a9')
    def check(ans, expected=R):
        return check_numeric(ans, expected)
    return {
        'question': f'{V}V at {I_str} \u2192 resistance?',
        'check': check,
        'answer_display': R_display,
    }


run_round(make_round4(), 'Round 4: Ohm\'s Law with Prefixes')

In [ ]:
# ---- Round 5: RC Time Constants ----

def make_round5():
    problems = []

    rc_combos = [
        # (R_val, R_str, C_val, C_str)
        (1e3, '1k\u03a9', 1e-6, '1\u00b5F'),
        (10e3, '10k\u03a9', 1e-6, '1\u00b5F'),
        (10e3, '10k\u03a9', 100e-9, '100nF'),
        (100e3, '100k\u03a9', 47e-9, '47nF'),
        (4.7e3, '4.7k\u03a9', 100e-9, '100nF'),
        (1e6, '1M\u03a9', 10e-12, '10pF'),
        (1e6, '1M\u03a9', 1e-6, '1\u00b5F'),
        (47e3, '47k\u03a9', 10e-9, '10nF'),
        (2.2e3, '2.2k\u03a9', 470e-9, '470nF'),
        (100e3, '100k\u03a9', 1e-6, '1\u00b5F'),
        (10e3, '10k\u03a9', 10e-9, '10nF'),
        (330e3, '330k\u03a9', 100e-9, '100nF'),
    ]

    chosen = random.sample(rc_combos, 8)
    for R, R_str, C, C_str in chosen:
        tau = R * C
        tau_display = format_eng(tau, 's')
        def check(ans, expected=tau):
            return check_numeric(ans, expected)
        problems.append({
            'question': f'\u03c4 = {R_str} \u00d7 {C_str} = ?',
            'check': check,
            'answer_display': tau_display,
        })

    return problems

run_round(make_round5(), 'Round 5: RC Time Constants')

In [ ]:
# ---- Round 6: Power Calculations ----

def make_round6():
    problems = []

    # P = V^2 / R
    for _ in range(3):
        V = random.choice([3.3, 5, 9, 12])
        R_val, R_str = random.choice([
            (100, '100\u03a9'), (220, '220\u03a9'), (470, '470\u03a9'),
            (1e3, '1k\u03a9'), (2.2e3, '2.2k\u03a9'), (10e3, '10k\u03a9'),
        ])
        P = V**2 / R_val
        P_display = format_eng(P, 'W')
        def check(ans, expected=P):
            return check_numeric(ans, expected)
        problems.append({
            'question': f'P = {V}V\u00b2 / {R_str} = ?',
            'check': check,
            'answer_display': P_display,
        })

    # P = VI
    for _ in range(3):
        V = random.choice([3.3, 5, 9, 12])
        I_val, I_str = random.choice([
            (1e-3, '1mA'), (5e-3, '5mA'), (10e-3, '10mA'),
            (20e-3, '20mA'), (50e-3, '50mA'), (100e-3, '100mA'),
        ])
        P = V * I_val
        P_display = format_eng(P, 'W')
        def check(ans, expected=P):
            return check_numeric(ans, expected)
        problems.append({
            'question': f'P = {V}V \u00d7 {I_str} = ?',
            'check': check,
            'answer_display': P_display,
        })

    # P = I^2 R
    for _ in range(2):
        I_val, I_str = random.choice([
            (10e-3, '10mA'), (20e-3, '20mA'), (50e-3, '50mA'),
            (100e-3, '100mA'),
        ])
        R_val, R_str = random.choice([
            (100, '100\u03a9'), (220, '220\u03a9'), (470, '470\u03a9'),
            (1e3, '1k\u03a9'),
        ])
        P = I_val**2 * R_val
        P_display = format_eng(P, 'W')
        def check(ans, expected=P):
            return check_numeric(ans, expected)
        problems.append({
            'question': f'P = ({I_str})\u00b2 \u00d7 {R_str} = ?',
            'check': check,
            'answer_display': P_display,
        })

    random.shuffle(problems)
    return problems

run_round(make_round6(), 'Round 6: Power Calculations')

In [ ]:
# ---- Round 7: Cutoff Frequencies ----

def make_round7():
    problems = []

    rc_combos = [
        (10e3, '10k\u03a9', 100e-9, '100nF'),
        (10e3, '10k\u03a9', 10e-9, '10nF'),
        (1e3, '1k\u03a9', 100e-9, '100nF'),
        (1e3, '1k\u03a9', 1e-6, '1\u00b5F'),
        (47e3, '47k\u03a9', 10e-9, '10nF'),
        (100e3, '100k\u03a9', 10e-9, '10nF'),
        (4.7e3, '4.7k\u03a9', 47e-9, '47nF'),
        (100e3, '100k\u03a9', 100e-9, '100nF'),
        (2.2e3, '2.2k\u03a9', 1e-6, '1\u00b5F'),
    ]

    chosen = random.sample(rc_combos, 6)
    for R, R_str, C, C_str in chosen:
        fc = 1 / (2 * math.pi * R * C)
        fc_display = format_eng(fc, 'Hz')
        def check(ans, expected=fc):
            return check_numeric(ans, expected, tolerance=0.05)
        problems.append({
            'question': f'f_c = 1/(2\u03c0 \u00d7 {R_str} \u00d7 {C_str}) \u2248 ?',
            'check': check,
            'answer_display': f'{fc_display} ({fc:.1f}Hz)',
        })

    return problems

run_round(make_round7(), 'Round 7: Cutoff Frequencies')

In [ ]:
# ---- Round 8: Mixed Boss Round ----

def make_round8():
    """12 problems drawn from all categories."""
    problems = []

    # 2 from unit conversion (harder ones)
    problems.append(_boss_conversion())
    problems.append(_boss_conversion())

    # 3 from Ohm's law
    problems.append(_ohm_find_I())
    problems.append(_ohm_find_V())
    problems.append(_ohm_find_R())

    # 2 from RC
    rc_pairs = [
        (22e3, '22k\u03a9', 47e-9, '47nF'),
        (330e3, '330k\u03a9', 22e-9, '22nF'),
        (4.7e3, '4.7k\u03a9', 1e-6, '1\u00b5F'),
        (1e6, '1M\u03a9', 100e-12, '100pF'),
    ]
    for R, R_str, C, C_str in random.sample(rc_pairs, 2):
        tau = R * C
        tau_display = format_eng(tau, 's')
        def check(ans, expected=tau):
            return check_numeric(ans, expected)
        problems.append({
            'question': f'\u03c4 = {R_str} \u00d7 {C_str} = ?',
            'check': check,
            'answer_display': tau_display,
        })

    # 2 from power
    V = random.choice([3.3, 5, 9])
    R = random.choice([220, 470, 1e3])
    P = V**2 / R
    def check_p1(ans, expected=P):
        return check_numeric(ans, expected)
    problems.append({
        'question': f'P = {V}V\u00b2 / {format_eng(R, chr(937))} = ?',
        'check': check_p1,
        'answer_display': format_eng(P, 'W'),
    })

    V2 = random.choice([5, 12])
    I2_val, I2_str = random.choice([(20e-3, '20mA'), (50e-3, '50mA'), (100e-3, '100mA')])
    P2 = V2 * I2_val
    def check_p2(ans, expected=P2):
        return check_numeric(ans, expected)
    problems.append({
        'question': f'P = {V2}V \u00d7 {I2_str} = ?',
        'check': check_p2,
        'answer_display': format_eng(P2, 'W'),
    })

    # 3 from cutoff frequency
    fc_pairs = [
        (10e3, '10k\u03a9', 47e-9, '47nF'),
        (1e3, '1k\u03a9', 470e-9, '470nF'),
        (100e3, '100k\u03a9', 1e-9, '1nF'),
        (4.7e3, '4.7k\u03a9', 100e-9, '100nF'),
    ]
    for R, R_str, C, C_str in random.sample(fc_pairs, 3):
        fc = 1 / (2 * math.pi * R * C)
        fc_display = format_eng(fc, 'Hz')
        def check_fc(ans, expected=fc):
            return check_numeric(ans, expected, tolerance=0.05)
        problems.append({
            'question': f'f_c = 1/(2\u03c0 \u00d7 {R_str} \u00d7 {C_str}) \u2248 ?',
            'check': check_fc,
            'answer_display': f'{fc_display} ({fc:.1f}Hz)',
        })

    random.shuffle(problems)
    return problems


def _boss_conversion():
    """A harder unit conversion problem."""
    templates = [
        ('0.0033\u00b5F in nF', 3.3, 'nF', 3.3e-9),
        ('2200pF in nF', 2.2, 'nF', 2.2e-9),
        ('0.47M\u03a9 in k\u03a9', 470, 'k\u03a9', 470e3),
        ('3300\u00b5A in mA', 3.3, 'mA', 3.3e-3),
        ('0.068\u00b5F in nF', 68, 'nF', 68e-9),
        ('56000pF in \u00b5F', 0.056, '\u00b5F', 56e-9),
    ]
    desc, expected_num, unit, expected_si = random.choice(templates)
    def check(ans, expected=expected_num):
        return check_numeric(ans, expected)
    return {
        'question': f'{desc}?',
        'check': check,
        'answer_display': f'{expected_num:g}{unit}',
    }


run_round(make_round8(), 'Round 8: Mixed Boss Round')

In [ ]:
# ---- Scoreboard ----

if not session_results:
    print('No rounds completed yet. Run the round cells above first!')
else:
    print(f'\n{"=" * 65}')
    print(f'{"SESSION SCOREBOARD":^65}')
    print(f'{"=" * 65}')
    print(f'{"Round":<35} {"Score":>7} {"Acc":>6} {"Time":>7} {"Per Q":>7}')
    print(f'{"-" * 65}')

    total_score = 0
    total_questions = 0
    total_time = 0
    worst_acc = 100
    worst_round = ''

    for r in session_results:
        total_score += r['score']
        total_questions += r['total']
        total_time += r['time']
        if r['accuracy'] < worst_acc:
            worst_acc = r['accuracy']
            worst_round = r['round']
        print(f'{r["round"]:<35} {r["score"]:>3}/{r["total"]:<3} '
              f'{r["accuracy"]:>5.0f}% {r["time"]:>6.1f}s {r["per_problem"]:>6.1f}s')

    overall_acc = total_score / total_questions * 100 if total_questions else 0
    print(f'{"-" * 65}')
    print(f'{"TOTAL":<35} {total_score:>3}/{total_questions:<3} '
          f'{overall_acc:>5.0f}% {total_time:>6.1f}s {total_time/total_questions:>6.1f}s')
    print()

    if worst_acc < 80:
        print(f'\u26a0  Weakest round: {worst_round} ({worst_acc:.0f}%) \u2014 try re-running it!')
    elif overall_acc >= 90:
        print('Strong performance across the board.')
    else:
        print(f'Weakest round: {worst_round} ({worst_acc:.0f}%) \u2014 consider re-running it.')

    print('\nRe-run any round cell for fresh problems and another attempt.')

---
## Quick Reference Card

### SI Prefixes

| Symbol | Name  | 10^n  |
|--------|-------|-------|
| p      | pico  | 10\u207b\u00b9\u00b2 |
| n      | nano  | 10\u207b\u2079  |
| \u00b5      | micro | 10\u207b\u2076  |
| m      | milli | 10\u207b\u00b3  |
| k      | kilo  | 10\u00b3   |
| M      | mega  | 10\u2076   |

### Prefix Product Shortcuts

| Multiply | Exponents | Result | Example |
|----------|-----------|--------|---------|
| k\u03a9 \u00d7 \u00b5F | 10\u00b3 \u00d7 10\u207b\u2076 | ms | 10k\u03a9 \u00d7 1\u00b5F = 10ms |
| k\u03a9 \u00d7 nF | 10\u00b3 \u00d7 10\u207b\u2079 | \u00b5s | 10k\u03a9 \u00d7 100nF = 1000\u00b5s = 1ms |
| M\u03a9 \u00d7 pF | 10\u2076 \u00d7 10\u207b\u00b9\u00b2 | \u00b5s | 1M\u03a9 \u00d7 10pF = 10\u00b5s |
| V \u00f7 k\u03a9 | 10\u2070 \u00f7 10\u00b3 | mA | 5V / 10k\u03a9 = 0.5mA |
| V \u00f7 M\u03a9 | 10\u2070 \u00f7 10\u2076 | \u00b5A | 5V / 1M\u03a9 = 5\u00b5A |

### Capacitor 3-Digit Code

Read as: first two digits \u00d7 10^(third digit) **in picofarads**.

| Code | pF | nF | \u00b5F |
|------|-----|------|------|
| 102  | 1000 | 1 | 0.001 |
| 103  | 10,000 | 10 | 0.01 |
| 104  | 100,000 | 100 | 0.1 |
| 105  | 1,000,000 | 1000 | 1 |
| 473  | 47,000 | 47 | 0.047 |